In [ ]:
from labda_parsers import Sens
from acti_motus import Features

from pyinstrument import Profiler

import logging

# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

df = Sens().from_file('../test_data/thigh.bin')
# df['acc_x'] = -df['acc_x']

# df.iloc[20000:200000] = 0
df.iloc[0:250000] = 0

profiler = Profiler()

with profiler:
    features = Features()
    extracted = features.extract(df)
    # timestamps, data = features._df_to_server(df)
    # sens = features.sens_extract(timestamps, data)

extracted.to_parquet('../temp/features.parquet')
profiler.write_html('../temp/features.html')

In [ ]:
from acti_motus.classification.thigh import Thigh
import pandas as pd
import logging
from acti_motus.classification.references import References


# Configure logging to see info messages
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

df = pd.read_parquet('../temp/features.parquet')
sensor = Thigh(vendor='Sens')

references = {
    'thigh': {
        'value': -0.201,
        'expires': '2024-09-03 12:05:51+00:00',
    },
    'trunk': {
        'value': [1.320, 0.247, -0.038],
        'expires': '2024-09-04 13:05:51+00:00',
    },
    'calibrations': [
        {
            'start': '2024-09-03 08:08:51+00:00',
            'end': '2024-09-03 08:09:11+00:00',
            'ttl': '24h',
        },
        {
            'start': '2024-09-02 10:36:31+00:00',
            'end': '2024-09-02 20:36:51+00:00',
            'ttl': '12h',
        },
        {
            'start': '2024-09-03 08:16:23+00:00',
            'end': '2024-09-03 10:17:03+00:00',
            'ttl': '12h',
        },
    ],
}

references = References(**references)
# references.remove_outdated_references(df)
df[['inclination', 'side_tilt', 'direction']] = sensor.get_angles(df)
non_wear = sensor.get_non_wear(df)
bouts = references.get_bouts(non_wear, 'thigh')
df, new_bouts = sensor.update_df_by_references(df, bouts)

new_bouts